In [115]:
import os
import pandas

# 1. Carregamento dos dados

Vamos começar nosso pipeline lendo o conjunto de dados fornecidos. Vamos aproveitar para ver o que tipo de informação esse conjunto de dados tem.

In [116]:
dataset_path = os.getenv("DATASET_PATH")
full_dataset_df = pandas.read_csv(dataset_path)
full_dataset_df

,product_id,seller_id,query,search_page,position,title,concatenated_tags,creation_date,price,weight,express_delivery,minimum_quantity,view_counts,order_counts,category
0,11394449,8324141,espirito santo,2,6,Mandala Espírito Santo,mandala mdf,2015-11-14 19:42:12,171.890000,1200.0,1,4,244,NaN,Decoração
1,15534262,6939286,cartao de visita,2,0,Cartão de Visita,cartao visita panfletos tag adesivos copos lon...,2018-04-04 20:55:07,77.670000,8.0,1,5,124,NaN,Papel e Cia
2,16153119,9835835,expositor de esmaltes,1,38,Organizador expositor p/ 70 esmaltes,expositor,2018-10-13 20:57:07,73.920006,2709.0,1,1,59,NaN,Outros
3,15877252,8071206,medidas lencol para berco americano,1,6,Jogo de Lençol Berço Estampado,t jogo lencol menino lencol berco,2017-02-27 13:26:03,118.770004,0.0,1,1,180,1.0,Bebê
4,15917108,7200773,adesivo box banheiro,3,38,ADESIVO BOX DE BANHEIRO,adesivo box banheiro,2017-05-09 13:18:38,191.810000,507.0,1,6,34,NaN,Decoração
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38502,16561714,9813770,nossa senhora de perolas,1,7,Imagem Nossa Senhora Aparecida em Perolas 25cm,senhora aparecida bebe perolas,2018-04-02 19:43:07,71.200000,706.0,1,4,315,15.0,Decoração
38503,12716324,6483096,lembrancinha personalizada dia dos pais,1,27,LEMBRANCINHA - DIA DOS PAIS,lembrancinhas,2018-07-10 11:41:08,14.650000,7.0,1,26,1288,17.0,Lembrancinhas
38504,972256,4840665,copo metalizado,1,3,Copos Metalizados - Rosé,despedida bianca metalizados xvdakaw lembranci...,2018-02-17 15:53:23,11.580000,25.0,1,104,306,NaN,Lembrancinhas
38505,7291315,6420917,festa do pijama menino,1,36,Kit Festa do Pijama Meninos,festa pijama,2016-08-21 22:33:59,131.330000,0.0,1,11,55,NaN,Lembrancinhas


Para entender melhor o problema, vamos ver quais são as categorias desse conjunto de dados, e qual a distribuição dessas categorias.

In [117]:
full_dataset_df["category"].unique()
full_dataset_df.groupby("category").product_id.count()

category
Bebê                   7026
Bijuterias e Jóias      951
Decoração              8846
Lembrancinhas         17759
Outros                 1148
Papel e Cia            2777
Name: product_id, dtype: int64

- parece que não precisamos fazer nenhuma modificação nos rótulos, não existem versões duplicadas com mudanças pequenas.
- todas categorias parecem ter um número razoável de amostras.

# 2. Transformação de dados

Vamos separar nossos dados em conjunto de treino, teste e validação.

In [118]:
def split_train_test_validate(full_df):
    shuffled_dataset_df = full_df.sample(frac=1).reset_index(drop=True)
    train_cut_idx = int(len(full_df) * .6)
    validation_cut_idx = train_cut_idx + int(len(full_df) * .2)
    return (
        full_df[:train_cut_idx],
        full_df[train_cut_idx:validation_cut_idx],
        full_df[validation_cut_idx:]
    )

train_data, test_data, validate_data = split_train_test_validate(full_dataset_df)
assert(len(full_dataset_df) == len(train_data) + len(test_data) + len(validate_data))

Agora vamos definir o dado que será input no nosso modelo. Como já comentamos antes, não parece ser necessário fazer nenhuma modificação nos rótulos (`category`), mas para o conjunto de características vamos precisar fazer algumas mudanças.

Por conhecimento prévio, e olhando a [descrição de cada coluna do conjunto de dados](https://github.com/elo7/data7_oss/tree/master/elo7-search), parece fazer sentido utilizar os campos de texto: `title`, com o título do produto, `concatenated_tags`, que são tags concatenadas, escolhidas pelo usuário, e `query` (?)

In [119]:
def concatenate_text_columns(dataframe):
    concatenated_text_information = dataframe["title"] + " " + dataframe["query"] + " " + dataframe["concatenated_tags"].fillna("")
    return concatenated_text_information

assert(concatenate_text_columns(train_data[:1])[0] == "Mandala Espírito Santo espirito santo mandala mdf")

Antes de criar nosso classificador, ainda precisamos transformar nossa feature em um vetor de valores númericos.

Decidimos fazer essa transformação porque pretendemos usar um classificador SVM. Além disso, parece ser padrão pra aprendizado de máquina em documentos de texto (veja: [Working With Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)).

Para fazer essa transformação, vamos usar o conjunto de textos concatenados como nosso corpus, e, a partir de uma ordanação arbitrária de palavras, construir vetores caracteríscos que indicam na i-ésima posição, a frequência da i-ésima palavra.

Obs: também testamos usar TF-IDF, mas a melhora no erro de teste foi menor que 1%, então decidimos usar apenas a frequência de palavras (TF) porque é uma transformação mais simples.

In [128]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


train_concatenated_text = concatenate_text_columns(train_data)
count_vectorizer = CountVectorizer()
count_vectorizer.fit(train_concatenated_text)
train_feature_wc = count_vectorizer.transform(train_concatenated_text)

tf_transformer = TfidfTransformer(use_idf=False).fit(train_feature_wc)
train_feature_tf = tf_transformer.transform(train_feature_wc)

def feature_extractor(dataframe):
    concatenated_text = concatenate_text_columns(dataframe)
    wc_vector = count_vectorizer.transform(concatenated_text)
    return tf_transformer.transform(wc_vector)


# 3. Modelagem

Por simplicidade, vamos usar um classificador da classe Support Vector Machine (SVM). Vamos usar um kernel linear, também por simplicidade.
Para selecionar o parâmetro de regularização C, vamos rodar um teste de validação cruzada com os dados de treinamento apenas.

In [136]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV


classifier = svm.SVC(kernel="linear")
parameters = {"C": [1, 10]}
cv_classifier = GridSearchCV(classifier, parameters)
cv_classifier.fit(train_feature_tf, train_data["category"])

print(cv_classifier.cv_results_)

{'mean_fit_time': array([19.72704096, 17.37211232]), 'std_fit_time': array([2.18840638, 1.49689006]), 'mean_score_time': array([3.72496014, 3.43935695]), 'std_score_time': array([0.46328979, 0.24394717]), 'param_C': masked_array(data=[1, 10],
             mask=[False, False],
       fill_value='?',
            dtype=object), 'params': [{'C': 1}, {'C': 10}], 'split0_test_score': array([0.88011253, 0.88595542]), 'split1_test_score': array([0.88400779, 0.8909327 ]), 'split2_test_score': array([0.87318762, 0.88227656]), 'split3_test_score': array([0.87881411, 0.88206016]), 'split4_test_score': array([0.87229437, 0.87532468]), 'mean_test_score': array([0.87768328, 0.8833099 ]), 'std_test_score': array([0.0043916 , 0.00512834]), 'rank_test_score': array([2, 1], dtype=int32)}


Olhando o `mean_test_score`, podemos ver que com `C = 10` tivemos uma pequena melhora no classificador, comparado a `C = 1`. Inclusive, em todos os splits feitos na validação cruzada, o valor de `C = 10` parece ter sido melhor

In [142]:
# best estimator
chosen_classifier = cv_classifier.best_estimator_

# test error
import numpy as np

test_feature_tf = feature_extractor(test_data)
predicted = chosen_classifier.predict(test_feature_tf)
np.mean(predicted == test_data["category"])

0.8923516426438125

# 4. Validação do modelo

Agora que já escolhemos nosso modelo, vamos validá-lo

In [143]:
validate_feature_tf = feature_extractor(validate_data)
predicted = chosen_classifier.predict(validate_feature_tf)
np.mean(predicted == validate_data["category"])

0.8962607115035056

Nosso modelo escolhido **acertou a categoria de 89% dos produtos** do conjunto de validação.

# 5. Exportação do modelo
Vamos usar um arquivo pickle para exportar nosso modelo